# PS4-4: Independent Components Analysis (ICA)

## Import Library

In [24]:
import numpy as np
import scipy.io.wavfile
import os

from scipy.misc import derivative

## Global Parameter

In [25]:
Fs = 11025

## Training ICA Model

### Training Algorithm

In [26]:
def update_W(W, x, learning_rate):
    """
    Perform a gradient ascent update on W using data element x and the provided learning rate.

    This function should return the updated W.

    Use the laplace distribution in this problem.

    Args:
        W: The W matrix for ICA
        x: A single data element
        learning_rate: The learning rate to use

    Returns:
        The updated W
    """

    # *** START CODE HERE ***
    # Ensure x is in 2D not 1D
    x = x.reshape(-1, 1) #Column vector

    # 1. Derivative of Log-Likelihood
    deri_ll = np.linalg.inv(W).T - np.sign(W @ x) @ x.T

    # 2. Update W
    W += learning_rate * deri_ll
    # *** END CODE HERE ***

    return W

In [27]:
def unmix(X, W):
    """
    Unmix an X matrix according to W using ICA.

    Args:
        X: The data matrix
        W: The W for ICA

    Returns:
        A numpy array S containing the split data
    """


    # *** START CODE HERE ***
    S = X @ W.T
    # *** END CODE HERE ***

    return S

In [28]:
def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

In [29]:
def load_data():
    mix = np.loadtxt('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS4/data/mix.dat')
    return mix

In [30]:
def save_W(W):
    np.savetxt('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set_Solution/PS4/output/W.txt',W)

In [31]:
def save_sound(audio, name):
    scipy.io.wavfile.write('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set_Solution/PS4/output/{}.wav'.format(name), Fs, audio)

In [32]:
def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1 , 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01 , 0.01, 0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    for lr in anneal:
        print(lr)
        rand = np.random.permutation(range(M))
        for i in rand:
            x = X[i]
            W = update_W(W, x, lr)

    return W

### Training Model

In [33]:
def main():
    # Seed the randomness of the simulation so this outputs the same thing each time
    np.random.seed(0)
    X = normalize(load_data())

    print(X.shape)

    for i in range(X.shape[1]):
        save_sound(X[:, i], 'mixed_{}'.format(i))

    W = unmixer(X)
    print(W)
    save_W(W)
    S = normalize(unmix(X, W))
    assert S.shape[1] == 5
    for i in range(S.shape[1]):
        if os.path.exists('split_{}'.format(i)):
            os.unlink('split_{}'.format(i))
        save_sound(S[:, i], 'split_{}'.format(i))

In [34]:
main()

(53442, 5)
Separating tracks ...
0.1
0.1
0.1
0.05
0.05
0.05
0.02
0.02
0.01
0.01
0.005
0.005
0.002
0.002
0.001
0.001
[[ 52.83418582  16.79558913  19.9407634  -10.19839844 -20.89724663]
 [ -9.93900225  -0.9813516   -4.68210573   8.04543479   1.79112857]
 [  8.31207077  -7.47564159  19.31468744  15.17352441 -14.32549324]
 [-14.66671561 -26.6440192    2.44079575  21.38118848  -8.42042467]
 [ -0.26929148  18.37388528   9.31215636   9.10251049  30.59383101]]
